In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime,timedelta

import os
path = os.getcwd()
if path[1:5] == "Home":
    %run /Home/siv30/wad005/master/GRB/src/paths/uib.ipynb
else:
    %run /Users/andreas/phys/master/GRB/src/paths/mac.ipynb


all UiB paths loaded


/Home/siv30/wad005/useful_py_functions/matplotlib_custom_style_and_macros.py:128: MatplotlibDeprecationWarning: Support for setting an rcParam that expects a str value to a non-str value is deprecated since 3.5 and support will be removed two minor releases later.
  rcParams['lines.marker'] = None


# NOTES
    -List of the triggers that we need to extract the bg data for:
        [GRB180720B, GRB181222B, GRB190305A, GRB210619B, GRB211211A]
        
    - Could the bg be the problem of the normalization constant?

# Loading trigger times

In [18]:
df_bg = pd.read_csv('trigger_times.txt',sep=' ')
df_bg.drop(columns='Unnamed: 6',axis=0,inplace=True)
bg_triggers = []
year,month,day,hour,minute,second = df_bg['year'].to_numpy(dtype=str),df_bg['month'].to_numpy(dtype=str), df_bg['day'].to_numpy(dtype=str),df_bg['hour'].to_numpy(dtype=str),df_bg['minute'].to_numpy(dtype=str),df_bg['second'].to_numpy(dtype=str)

for i in np.arange(0,len(year)):
    try:
        dt_obj = datetime.strptime(year[i]+'/'+month[i]+'/'+day[i]+' '+hour[i]+':'+minute[i]+':'+second[i],'%Y/%m/%d %H:%M:%S.%f')
        bg_triggers.append(dt_obj)
    except ValueError:
        if second[i] == '60.0':
            second[i] = '59.999'
            dt_obj = datetime.strptime(year[i]+'/'+month[i]+'/'+day[i]+' '+hour[i]+':'+minute[i]+':'+second[i],'%Y/%m/%d %H:%M:%S.%f')
            bg_triggers.append(dt_obj)
print('All arr lengths match: ',len(year)==len(month)==len(day)==len(hour)==len(minute)==len(second)==len(bg_triggers))

All arr lengths match:  True


In [19]:
df_trigtimes

,year,month,day,hour,minute,second
0,2018,6,1,5,10,4.3
1,2018,6,1,5,12,2.8
2,2018,6,1,5,15,53.1
3,2018,6,1,5,16,35.4
4,2018,6,1,9,54,34.9
...,...,...,...,...,...,...
692439,2022,11,1,23,30,26.7
692440,2022,11,1,23,36,15.8
692441,2022,11,1,23,44,45.8
692442,2022,11,1,23,46,58.5


In [4]:
#Creating datetime object from all rows, then computing timedelta for the relevant entries based on search criteria

In [6]:
#not optimal to compute the timedelta for all 600k rows, but there is only a handful of candidates we need to do it for...
#approx 2 min for whole df to compute...
# 982x faster to only compute for the relevant DAY

In [12]:
ASIM_trig_str = '2019-03-05 13:05:19.816'
ASIM_trig_dt = datetime.strptime(ASIM_trig_str,'%Y-%m-%d %H:%M:%S.%f')

## Convert string/datetime

In [2]:
def convert_str_datetime(dt_string=None):
    dt_obj = datetime.strptime(dt_string,'%Y-%m-%d %H:%M:%S.%f')
    return dt_obj

## find background match

In [37]:
def find_bg_match(ASIM_trig_dt=None,bg_triggers=None, time_limit=120.0):
    '''ASIM_trig_dt <datetime> trigger time for ASIM
    bg_triggers <np.arr> contains datetime objects for trigger data for background
    time_limit <float> limit for the difference between ASIM_trig_dt & datetime_trig. Default 120 s
    
    return <np.array>,<np.array> first array contains time difference and second array is index for trigger in datetime_trig
    '''

    trig_day = []
    index_trig_day = []

    for i,dt in enumerate(bg_triggers):
        if dt.year == ASIM_trig_dt.year and dt.month==ASIM_trig_dt.month and dt.day == ASIM_trig_dt.day:
            diff_day = timedelta.total_seconds(bg_triggers[i]-ASIM_trig_dt)
            trig_day.append(diff_day)
            index_trig_day.append(i)
        else:
            continue
    trig_day = np.array(trig_day)
    mask_close = np.where(np.abs(trig_day)<=time_limit)[0]
    
    return trig_day[mask_close], index_trig_day[mask_close[0]:mask_close[-1]+1]
    

## List of trigger times 
    [GRB180720B, GRB181222B, GRB190305A, GRB210619B, GRB211211A]
    
    Looks like we have to go hunting for bg after the burst for several of the triggers

In [6]:
T0_180720,T0_181222,T0_190305,T0_210619, T0_211211 = '2018-07-20 14:21:47.389', '2018-12-22 20:11:36.576','2019-03-05 13:05:19.816','2021-06-19 23:59:24.915', '2021-12-11 13:10:01.323'
T0_arr = [T0_180720,T0_181222,T0_190305,T0_210619, T0_211211]

In [25]:
i,offset = [],[]
for t0 in T0_arr:
    t0_obj = convert_str_datetime(t0)
    time_offset, idxs =  find_bg_match(ASIM_trig_dt=t0_obj,bg_triggers=bg_triggers)
    print(t0_obj, time_offset)
    offset.append(time_offset)
    i.append(idxs)

2018-07-20 14:21:47.389000 [ 0.311  3.011  8.211  9.911 21.611]
2018-12-22 20:11:36.576000 [0.924]
2019-03-05 13:05:19.816000 [-91.216   0.284  81.584]
2021-06-19 23:59:24.915000 [ 0.585  1.585  2.585  3.585  8.985  9.985 11.185 12.185 13.285 14.885]
2021-12-11 13:10:01.323000 [7.7000e-02 1.0770e+00 4.2770e+00 5.2770e+00 1.4777e+01 1.6077e+01
 1.7677e+01 1.8977e+01 8.0777e+01]


In [35]:
for ii in i:
    print(df_trigtimes.iloc[ii]) #~ 1.5 min before trigger

       year  month  day  hour  minute  second
17642  2018      7   20    14      21    47.7
17643  2018      7   20    14      21    50.4
17644  2018      7   20    14      21    55.6
17645  2018      7   20    14      21    57.3
17646  2018      7   20    14      22     9.0
        year  month  day  hour  minute  second
111933  2018     12   22    20      11    37.5
        year  month  day  hour  minute  second
157172  2019      3    5    13       3    48.6
157173  2019      3    5    13       5    20.1
157174  2019      3    5    13       6    41.4
        year  month  day  hour  minute  second
362535  2021      6   19    23      59    25.5
362536  2021      6   19    23      59    26.5
362537  2021      6   19    23      59    27.5
362538  2021      6   19    23      59    28.5
362539  2021      6   19    23      59    33.9
362540  2021      6   19    23      59    34.9
362541  2021      6   19    23      59    36.1
362542  2021      6   19    23      59    37.1
362543  2021      6

In [38]:
df_bg.iloc[438668-3:438668+3]

,year,month,day,hour,minute,second
438665,2021,12,11,12,38,25.6
438666,2021,12,11,12,44,45.9
438667,2021,12,11,12,50,0.4
438668,2021,12,11,13,10,1.4
438669,2021,12,11,13,10,2.4
438670,2021,12,11,13,10,5.6


In [36]:
find_bg_match(ASIM_trig_dt=convert_str_datetime(T0_211211),bg_triggers=bg_triggers[438667:-2:438667+2])

-1200.923


IndexError: index 0 is out of bounds for axis 0 with size 0

In [34]:
#get data for this trigger using read_matlab_file

#problem accessing the relevant cdf file.. old script works for unzipped files, while now all the cdf
#cdf files have been .gz zipped

#can improve the script be unzipping, but then; where to store the files? -->temp folder on the server
#or stored locally

#ask David tomorrow if he have a good script for reading the files and generating a matlab file
#containing the data